In [35]:
#import necessary packages
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import matplotlib.colors as mcolors
import re

In [36]:
#import necessary packages for further word processing
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(2020)
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /global/homes/d/dsmorrow/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [38]:
#read in csv saved from python; see lemmatized_all.py
#this csv includes text that only includes abbreviations, medications and misspellings
NED = pd.read_csv("No_English_Dictionary.csv", dtype=str)
NED.head()

,Unnamed: 0,ND_string2
0,0,"['allergi', 'percocet', 'lf', 'hemodialysi', '..."
1,1,"['allergi', 'percocet', 'lf', 'lastnam', 'esrd..."
2,2,"['allergi', 'percocet', 'lf', 'micu', 'sle', '..."
3,3,"['allergi', 'percocet', 'lf', 'hemodialysi', '..."
4,4,"['allergi', 'percocet', 'namepattern', 'hemodi..."


In [49]:
words = set(nltk.corpus.words.words())
def preprocess2(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and token not in words:
            result.append(token)
    return result

In [50]:
processed_docs2 = NED['ND_string2'].map(preprocess2)

In [51]:
#create dictionary of words
dictionary = gensim.corpora.Dictionary(processed_docs2)
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 aliskiren
1 allergi
2 amp
3 anicter
4 antibodi
5 anticardiolipin
6 anticoagul
7 autoimmun
8 baselin
9 bp
10 cerebrovascular


In [52]:
#filter out words that appear in less than (15) documents
#only keep the first 10000
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [54]:
#go through each document and report words and occurrences using doc2box for token id and amount
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs2]

In [68]:
#determine the TF-IDF scores or weight of a word within documents
from gensim import corpora, models
tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]
from pprint import pprint
for doc in corpus_tfidf:
    pprint(sorted(doc, key = lambda x: x[1]))
    break

[(111, 0.014824932283466754),
 (101, 0.015331008365657895),
 (60, 0.01701150571240441),
 (1, 0.017066553563346427),
 (63, 0.01720501358485213),
 (24, 0.017687606155084624),
 (95, 0.018922226688739562),
 (102, 0.019085287114207792),
 (46, 0.01946364199676075),
 (25, 0.01957405138039286),
 (79, 0.019680071704618662),
 (109, 0.019988752104761188),
 (113, 0.020086447203989666),
 (82, 0.020963733503304133),
 (64, 0.021228161663913055),
 (54, 0.02268044277953536),
 (97, 0.023498654611390484),
 (33, 0.0238130154015657),
 (35, 0.024037646095580328),
 (87, 0.024119027207548598),
 (23, 0.024691545880756817),
 (132, 0.02492167824234219),
 (49, 0.024993349927011913),
 (72, 0.02539866471739196),
 (39, 0.025463723567176487),
 (38, 0.027951286606202407),
 (43, 0.02799199777374769),
 (96, 0.029027967504815202),
 (17, 0.029874728760229143),
 (47, 0.030479058068940076),
 (32, 0.03106508874810856),
 (51, 0.03176116828983889),
 (130, 0.03183893374872267),
 (6, 0.03198683787419499),
 (107, 0.03198783596611

In [89]:
#top ten weighted words
print(dictionary[80] ,"," + dictionary[125] ,"," + dictionary[105] ,"," + dictionary[89] ,"," + dictionary[15] ,"," + dictionary[0] ,"," + dictionary[131],"," + dictionary[57],"," + dictionary[48],"," + dictionary[117])

mg ,transderm ,qwed ,nifedipin ,clonidin ,aliskiren ,wednesday ,hydralazin ,hd ,sle


In [90]:
#these weighted words have alot to do with medications
#mg = milligrams
#transderm = transdermal - relating to or denoting the application of a medicine or drug through the skin
#qwed = 
#nifedipine = Antihypertensive drug and Calcium channel blocker
#clonidine = Sedative and Antihypertensive drug
#aliskiren = Antihypertensive drug
#hd
#sle 
#high blood pressure treatment 

In [101]:
#top weighted words in document 30
tf_obj = corpus_tfidf[30]
sorted(tf_obj, key=lambda x: x[1], reverse=True)
n_terms = 5

top_terms = []
for obj in sorted(tf_obj, key=lambda x: x[1], reverse=True)[:n_terms]:
    top_terms.append("{0:s} ({1:01.03f})".format(dictionary[obj[0]], obj[1]))

print(top_terms)

['labetalol (0.361)', 'mg (0.316)', 'prednison (0.240)', 'nitropast (0.234)', 'nicardipin (0.185)']


In [103]:
#top weighted words in document 747
tf_obj = corpus_tfidf[747]
sorted(tf_obj, key=lambda x: x[1], reverse=True)
n_terms = 5

top_terms = []
for obj in sorted(tf_obj, key=lambda x: x[1], reverse=True)[:n_terms]:
    top_terms.append("{0:s} ({1:01.03f})".format(dictionary[obj[0]], obj[1]))

print(top_terms)

['labetalol (0.338)', 'sbo (0.330)', 'hd (0.277)', 'sle (0.263)', 'ml (0.251)']


In [104]:
#top weighted words in document 382
tf_obj = corpus_tfidf[382]
sorted(tf_obj, key=lambda x: x[1], reverse=True)
n_terms = 5

top_terms = []
for obj in sorted(tf_obj, key=lambda x: x[1], reverse=True)[:n_terms]:
    top_terms.append("{0:s} ({1:01.03f})".format(dictionary[obj[0]], obj[1]))

print(top_terms)

['med (0.409)', 'zofran (0.399)', 'pt (0.302)', 'pd (0.296)', 'orderd (0.273)']
